In [9]:
# Langchain의 챗 모델과 다양한 문서 로더, 텍스트 분할기, 임베딩, 벡터 스토어 등을 임포트합니다.
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

# OpenAI의 챗 모델을 낮은 온도 설정과 함께 초기화합니다.
llm = ChatOpenAI(
    temperature=0.1,
)

# 캐시 디렉토리를 설정합니다.
cache_dir = LocalFileStore("./.cache/")

# 텍스트를 적절한 크기의 청크로 분할하는 분할기를 설정합니다.
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

# 비구조화된 파일로부터 문서를 로드합니다.
loader = UnstructuredFileLoader("./files/chapter_one.docx")

# 로더를 사용하여 문서를 로드하고 분할합니다.
docs = loader.load_and_split(text_splitter=splitter)

# OpenAI 임베딩을 초기화합니다.
embeddings = OpenAIEmbeddings()

# 캐시를 이용하여 임베딩을 백업합니다.
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

# FAISS 벡터 스토어를 문서로부터 생성합니다.
vectorstore = FAISS.from_documents(docs, cached_embeddings)

# 벡터 스토어를 검색기로 변환합니다.
retriver = vectorstore.as_retriever()

# 대화 프롬프트 템플릿을 설정합니다.
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system", 
            "도우미로서 주어진 문맥을 이용해 질문에 답하십시오. 모르는 경우 모르겠다고만 말하고, 추측하지 마십시오: \n\n{context}"),
        (
            "human", 
            "{question}"
        ),
    ]
)

# 체인을 구성하여 문맥, 질문, 추가 정보를 처리한 후 결과를 반환합니다.
chain = (
    {
        "context": retriver, 
        "question": RunnablePassthrough(), 
        "extra": RunnablePassthrough()
    } 
    | prompt 
    | llm
)

# 체인을 활성화하여 'Victory Mansions'에 대해 설명을 요청합니다.
chain.invoke("Describe Victory Mansions")


AIMessage(content='Victory Mansions is a building where Winston Smith resides. It is a run-down apartment complex with a faulty elevator, gritty dust, and a hallway that smells of boiled cabbage and old rag mats. The flat where Winston lives is on the seventh floor, and the building is poorly maintained due to the ongoing economy drive in preparation for Hate Week. The building is described as having a gritty and unpleasant atmosphere.')

[1. 문서 로더 초기화] -- UnstructuredFileLoader
  |
  |-> "./files/chapter_one.docx" 파일 로드
  |   설명: UnstructuredFileLoader는 구조화되지 않은 문서를 로드하는 데 사용됩니다. 여기서는 docx 파일을 로드합니다.
  |
[2. 텍스트 분할기 초기화] -- CharacterTextSplitter.from_tiktoken_encoder
  |   separator="\n", chunk_size=600, chunk_overlap=100
  |   설명: 텍스트를 지정된 크기로 분할하여 처리할 수 있도록 합니다. 여기서는 문자 기반 분할 방식을 사용합니다.
  |
  |-> 문서 분할
  |   설명: 로드된 문서를 위에서 정의한 분할기를 사용해 분할합니다.
  |
[3. 임베딩 및 캐싱] -- OpenAIEmbeddings & CacheBackedEmbeddings
  |
  |-> OpenAI 임베딩 생성
  |   설명: OpenAI 임베딩을 사용하여 문서의 텍스트를 벡터로 변환합니다.
  |
  |-> 캐시 디렉토리 설정: LocalFileStore("./.cache/")
  |   설명: 임베딩 결과를 캐시하기 위한 로컬 파일 스토어를 설정합니다.
  |
  |-> 캐시된 임베딩 생성
  |   설명: 임베딩을 캐시하여 재사용을 용이하게 하고, 처리 시간을 단축시킵니다.
  |
[4. 벡터 스토어 및 검색기 초기화] -- FAISS
  |
  |-> FAISS 벡터 스토어 생성
  |   설명: 캐시된 임베딩을 기반으로 FAISS 벡터 스토어를 생성합니다. 이를 통해 효율적인 유사도 검색이 가능해집니다.
  |
  |-> 검색기로 변환
  |   설명: 생성된 벡터 스토어를 사용하여 문서 내에서 정보를 검색할 수 있는 검색기를 초기화합니다.
  |
[5. 프롬프트 템플릿 및 체인 구성] -- ChatPromptTemplate & Chain
  |
  |-> 프롬프트 템플릿 설정
  |   설명: 사용자 질문과 시스템 응답을 포맷팅하기 위한 템플릿을 설정합니다.
  |
  |-> 체인 구성
  |   설명: 검색 결과, 사용자 질문, 그리고 추가적인 처리를 포함한 전체 처리 과정을 정의하는 체인을 구성합니다.
  |
[6. 체인 실행 및 질문 처리] -- chain.invoke("Describe Victory Mansions")
    설명: 구성된 체인을 사용하여 "Describe Victory Mansions"에 대한 질문을 처리하고, 관련 정보를 검색하여 응답을 생성합니다.


이 코드는 Langchain 라이브러리를 사용하여 대화형 프롬프트를 통해 문서의 내용을 기반으로 질문에 답변하는 시스템을 구축하는 과정을 보여줍니다. 여기서는 문서를 로드하고 분할하여 임베딩을 생성한 다음, FAISS를 이용해 벡터 스토어를 만들고, 이를 검색기로 사용합니다. 대화형 프롬프트는 시스템이 검색한 문맥을 기반으로 질문에 답변하도록 유도합니다. 이 과정은 복잡한 체이닝 메커니즘을 통해 실행되며, 결과적으로 사용자의 질문에 대한 답을 제공합니다.